# AquaCrop Simulation: Rainfed vs Irrigation Comparison

This notebook demonstrates a comparison of alfalfa growth under rainfed conditions versus irrigated conditions in Ottawa using the AquaCrop Python API.

## 1. Import Required Libraries

In [ ]:
%pip install pyaquacrop

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, datetime

# Import AquaCrop components
from aquacrop import AquaCrop, Crop, Soil, SoilLayer, Weather, FieldManagement, Irrigation

# Import templates for Ottawa
from aquacrop.templates import (
    ottawa_alfalfa, ottawa_sandy_loam, ottawa_temperatures, ottawa_rain, 
    ottawa_eto, manuloa_co2_records, ottawa_management
)

## 2. Set up Output Directories

In [ ]:
# Create output directories
output_dir_rainfed = "outputs/rainfed_simulation"
output_dir_irrigated = "outputs/irrigated_simulation"
charts_dir = "output_charts"

# Make directories if they don't exist
os.makedirs(output_dir_rainfed, exist_ok=True)
os.makedirs(output_dir_irrigated, exist_ok=True)
os.makedirs(charts_dir, exist_ok=True)

## 3. Define Simulation Period for Multiple Years

In [ ]:
# Define the simulation periods for multiple years
simulation_periods = [
    {
        "start_date": date(2014, 5, 21),
        "end_date": date(2014, 10, 31),
        "planting_date": date(2014, 5, 21),
        "is_seeding_year": True,
    },
    {
        "start_date": date(2015, 4, 1),
        "end_date": date(2015, 10, 31),
        "planting_date": date(2015, 4, 1),
        "is_seeding_year": False,  # This is the second year, not seeding year
    },
    {
        "start_date": date(2016, 4, 1),
        "end_date": date(2016, 10, 31),
        "planting_date": date(2016, 4, 1),
        "is_seeding_year": False,  # This is the third year, not seeding year
    }
]

# Extract years for later use in charts
simulation_years = [period["start_date"].year for period in simulation_periods]

## 4. Create Climate Object

In [ ]:
# Create a modified Ottawa rainfall dataset with reduced precipitation
def create_reduced_rainfall():
    # Make a copy of the original Ottawa rainfall data
    reduced_ottawa_rain = ottawa_rain.copy()
    
    # Reduce rainfall by 50% to create moderately dry conditions
    # Changed from 10% to 50% to ensure some growth but still demonstrate drought effects
    reduced_ottawa_rain = [rain * 0.5 for rain in reduced_ottawa_rain]
    
    print(f"Original rainfall total: {sum(ottawa_rain):.1f} mm")
    print(f"Reduced rainfall total: {sum(reduced_ottawa_rain):.1f} mm")
    print(f"Reduction: {100 - (sum(reduced_ottawa_rain)/sum(ottawa_rain)*100):.1f}%")
    
    return reduced_ottawa_rain

# Create reduced rainfall data
reduced_ottawa_rain = create_reduced_rainfall()

# Create climate object using the Ottawa templates
climate = Weather(
    location="Ottawa",
    temperatures=ottawa_temperatures,
    eto_values=ottawa_eto,
    rainfall_values=reduced_ottawa_rain,
    record_type=1,  # Daily records
    first_day=1,
    first_month=1,
    first_year=2014,
    co2_records=manuloa_co2_records,
)

## 5. Create Irrigation Schedule for the Irrigated Scenario

In [ ]:
# Create irrigation schedule (applied every 14 days from planting)
irrigation_events = []
for i in range(1, 10):  # 9 irrigation events
    day = i * 14  # Every 14 days
    irrigation_events.append({'day': day, 'depth': 30, 'ec': 0.0})  # 30 mm depth

# Create irrigation object
irrigation = Irrigation(
    name="Sprinkler Fixed Schedule",
    description="Sprinkler irrigation with fixed schedule every 14 days",
    params={
        'irrigation_method': 1,  # Sprinkler
        'surface_wetted': 100,
        'irrigation_mode': 1,  # Specification of events
        'reference_day': -9,  # Reference day is onset of growing period
        'irrigation_events': irrigation_events
    }
)

## 6. Run Rainfed Simulation

In [ ]:
# Create and run rainfed simulation (no irrigation)
simulation_rainfed = AquaCrop(
    simulation_periods=simulation_periods,
    crop=ottawa_alfalfa,
    soil=ottawa_sandy_loam,
    management=ottawa_management,
    climate=climate,
    working_dir=output_dir_rainfed,
    need_daily_output=True,
    need_seasonal_output=True,
    need_harvest_output=True
)

# Run the simulation
results_rainfed = simulation_rainfed.run()
print("Rainfed simulation completed!")

## 7. Run Irrigated Simulation

In [ ]:
# Create and run irrigated simulation
simulation_irrigated = AquaCrop(
    simulation_periods=simulation_periods,
    crop=ottawa_alfalfa,
    soil=ottawa_sandy_loam,
    management=ottawa_management,
    irrigation=irrigation,  # Add irrigation
    climate=climate,
    working_dir=output_dir_irrigated,
    need_daily_output=True,
    need_seasonal_output=True,
    need_harvest_output=True
)

# Run the simulation
results_irrigated = simulation_irrigated.run()
print("Irrigated simulation completed!")

## 8. Display Seasonal Results

In [ ]:
# Display seasonal results
print("\nRainfed Seasonal Results:")
display(results_rainfed['season'])

print("\nIrrigated Seasonal Results:")
display(results_irrigated['season'])

## 9. Process Daily Results for Visualization

In [ ]:
# Process daily results
def process_daily_results(results):
    daily_results = results['day']
    
    # Check if we have a dictionary (multiple runs) or a single DataFrame
    if isinstance(daily_results, dict):
        # For multiple runs, get the first run's dataframe
        run_key = list(daily_results.keys())[0]
        return daily_results[run_key]
    else:
        # Single DataFrame case
        return daily_results

# Extract daily dataframes
daily_rainfed = process_daily_results(results_rainfed)
daily_irrigated = process_daily_results(results_irrigated)

# Print column names to understand the data structure
print("Available data columns:", daily_rainfed.columns.tolist()[:20], "...")  # Show first 20 columns

## 10. Create Date Objects for Plotting

In [ ]:
# Create date objects for both simulations
def create_dates(df):
    dates = []
    valid_indices = []
    
    for i in range(len(df)):
        try:
            day = int(df['Day'].iloc[i])
            month = int(df['Month'].iloc[i])
            year = int(df['Year'].iloc[i])
            dates.append(datetime(year, month, day))
            valid_indices.append(i)
        except (ValueError, TypeError) as e:
            # Handle any invalid date values
            print(f"Invalid date at index {i}: {e}")
    
    return dates, valid_indices

# Get valid dates and indices
rainfed_dates, rainfed_indices = create_dates(daily_rainfed)
irrigated_dates, irrigated_indices = create_dates(daily_irrigated)

print(f"Valid dates for rainfed: {len(rainfed_dates)}")
print(f"Valid dates for irrigated: {len(irrigated_dates)}")

## 11. Visualize Biomass Comparison

In [ ]:
# Check if 'Biomass' column exists
if 'Biomass' in daily_rainfed.columns and 'Biomass' in daily_irrigated.columns:
    # Extract biomass values for valid dates
    rainfed_biomass = [daily_rainfed['Biomass'].iloc[i] for i in rainfed_indices]
    irrigated_biomass = [daily_irrigated['Biomass'].iloc[i] for i in irrigated_indices]
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot biomass for both simulations
    ax.plot(rainfed_dates, rainfed_biomass, 'b-', linewidth=2, label='Rainfed')
    ax.plot(irrigated_dates, irrigated_biomass, 'g-', linewidth=2, label='Irrigated')
    
    # Add grid and labels
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Biomass (ton/ha)', fontsize=12)
    ax.set_title('Alfalfa Biomass Accumulation: Rainfed vs Irrigated', fontsize=14)
    
    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    fig.autofmt_xdate()  # Rotate date labels for better readability
    
    # Add irrigation events as vertical lines for the first year only
    for event in irrigation_events:
        event_date = simulation_periods[0]['planting_date'] + pd.Timedelta(days=event['day'])
        event_datetime = datetime(event_date.year, event_date.month, event_date.day)
        ax.axvline(x=event_datetime, color='c', linestyle='--', alpha=0.5)
    
    # Add legend and annotation
    ax.legend(loc='upper left')
    plt.figtext(0.15, 0.02, "Cyan vertical lines indicate irrigation events (30mm each) in first year", 
               ha="left", fontsize=10, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    
    # Save figure
    fig_path = os.path.join(charts_dir, 'biomass_comparison_multi_year.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"Saved biomass comparison chart to {fig_path}")
    
    plt.show()
else:
    print("Biomass column not found in one or both datasets")
    print(f"Rainfed columns: {daily_rainfed.columns.tolist()[:10]}")
    print(f"Irrigated columns: {daily_irrigated.columns.tolist()[:10]}")

## 12. Visualize Canopy Cover Comparison

In [ ]:
# Check if 'CC' column exists
if 'CC' in daily_rainfed.columns and 'CC' in daily_irrigated.columns:
    # Extract canopy cover values for valid dates
    rainfed_cc = [daily_rainfed['CC'].iloc[i] for i in rainfed_indices]
    irrigated_cc = [daily_irrigated['CC'].iloc[i] for i in irrigated_indices]
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot canopy cover for both simulations
    ax.plot(rainfed_dates, rainfed_cc, 'b-', linewidth=2, label='Rainfed')
    ax.plot(irrigated_dates, irrigated_cc, 'g-', linewidth=2, label='Irrigated')
    
    # Add grid and labels
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Canopy Cover (%)', fontsize=12)
    ax.set_title('Alfalfa Canopy Cover: Rainfed vs Irrigated', fontsize=14)
    
    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    fig.autofmt_xdate()  # Rotate date labels for better readability
    
    # Add irrigation events as vertical lines for the first year only
    for event in irrigation_events:
        event_date = simulation_periods[0]['planting_date'] + pd.Timedelta(days=event['day'])
        event_datetime = datetime(event_date.year, event_date.month, event_date.day)
        ax.axvline(x=event_datetime, color='c', linestyle='--', alpha=0.5)
    
    # Add legend and annotation
    ax.legend(loc='lower right')
    plt.figtext(0.15, 0.02, "Cyan vertical lines indicate irrigation events (30mm each) in first year", 
               ha="left", fontsize=10, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    
    # Save figure
    fig_path = os.path.join(charts_dir, 'canopy_cover_comparison_multi_year.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"Saved canopy cover comparison chart to {fig_path}")
    
    plt.show()
else:
    print("CC column not found in one or both datasets")

## 13. Create Water Balance Comparison Table

In [ ]:
# Get season results from both simulations
season_rainfed = results_rainfed['season']
season_irrigated = results_irrigated['season']

# Helper function to safely get value from seasonal data
def get_value(df, column, row_index=0, default=0):
    if column in df.columns:
        if row_index < len(df[column]):
            return df[column].values[row_index]
        else:
            print(f"Warning: Row index {row_index} out of bounds for column {column}")
            return default
    else:
        print(f"Warning: {column} not found in seasonal data")
        return default

# Check how many seasons we have in the data
num_seasons = len(season_rainfed) if isinstance(season_rainfed, pd.DataFrame) else 0
print(f"Number of seasons in data: {num_seasons}")

# Create a multi-season comparison DataFrame
multi_season_data = []

# Process each season
for season_idx in range(num_seasons):
    year = simulation_years[season_idx]
    
    # Metrics for this season
    season_data = {
        'Year': year,
        'Treatment': 'Rainfed',
        'Total Biomass (ton/ha)': get_value(season_rainfed, 'Biomass', season_idx),
        'Total Yield (ton/ha)': get_value(season_rainfed, 'Y(dry)', season_idx),
        'Total Rainfall (mm)': get_value(season_rainfed, 'Rain', season_idx),
        'Total Irrigation (mm)': 0,  # No irrigation for rainfed
        'Total Evapotranspiration (mm)': get_value(season_rainfed, 'ET', season_idx, 
                                                  get_value(season_rainfed, 'E', season_idx) + 
                                                  get_value(season_rainfed, 'Tr', season_idx)),
        'Total Transpiration (mm)': get_value(season_rainfed, 'Tr', season_idx),
        'Water Productivity - Biomass (kg/m³)': get_value(season_rainfed, 'WPet', season_idx),
        'Water Productivity - Yield (kg/m³)': get_value(season_rainfed, 'WPet', season_idx, 0) * 
                                            (get_value(season_rainfed, 'Y(dry)', season_idx) / 
                                             get_value(season_rainfed, 'Biomass', season_idx, 1))
    }
    multi_season_data.append(season_data)
    
    # Add irrigated data for the same season
    season_data = {
        'Year': year,
        'Treatment': 'Irrigated',
        'Total Biomass (ton/ha)': get_value(season_irrigated, 'Biomass', season_idx),
        'Total Yield (ton/ha)': get_value(season_irrigated, 'Y(dry)', season_idx),
        'Total Rainfall (mm)': get_value(season_irrigated, 'Rain', season_idx),
        'Total Irrigation (mm)': get_value(season_irrigated, 'Irri', season_idx),
        'Total Evapotranspiration (mm)': get_value(season_irrigated, 'ET', season_idx,
                                                  get_value(season_irrigated, 'E', season_idx) + 
                                                  get_value(season_irrigated, 'Tr', season_idx)),
        'Total Transpiration (mm)': get_value(season_irrigated, 'Tr', season_idx),
        'Water Productivity - Biomass (kg/m³)': get_value(season_irrigated, 'WPet', season_idx),
        'Water Productivity - Yield (kg/m³)': get_value(season_irrigated, 'WPet', season_idx, 0) * 
                                            (get_value(season_irrigated, 'Y(dry)', season_idx) / 
                                             get_value(season_irrigated, 'Biomass', season_idx, 1))
    }
    multi_season_data.append(season_data)

# Create a DataFrame from the collected data
multi_season_df = pd.DataFrame(multi_season_data)

# Display the multi-season comparison
display(multi_season_df)

# Save to CSV
csv_path = os.path.join(charts_dir, 'multi_season_comparison.csv')
multi_season_df.to_csv(csv_path, index=False)
print(f"Saved multi-season comparison data to {csv_path}")

# Additionally, create a pivot table for easier year-by-year comparison
pivot_df = multi_season_df.pivot_table(
    index='Year', 
    columns='Treatment',
    values=['Total Biomass (ton/ha)', 'Total Yield (ton/ha)', 'Total Rainfall (mm)', 
            'Total Irrigation (mm)', 'Total Evapotranspiration (mm)', 'Total Transpiration (mm)',
            'Water Productivity - Biomass (kg/m³)', 'Water Productivity - Yield (kg/m³)']
)

# Display the pivot table
display(pivot_df)

# Save pivot table to CSV
pivot_csv_path = os.path.join(charts_dir, 'season_pivot_comparison.csv')
pivot_df.to_csv(pivot_csv_path)
print(f"Saved pivot table comparison to {pivot_csv_path}")

## 14. Visualize Water Balance Components Across Multiple Years

In [ ]:
# Create bar chart for water balance components across multiple years

# Define the metrics we want to visualize
water_balance_metrics = ['Total Rainfall (mm)', 'Total Irrigation (mm)', 
                         'Total Evapotranspiration (mm)', 'Total Transpiration (mm)']

# Extract data from multi_season_df instead of the undefined comparison_df
# We'll use the pivot_df that was created at the end of section 13
# This assumes pivot_df has been created correctly

# Create lists to store values for each year
rainfed_yearly_values = []
irrigated_yearly_values = []

# Organize data by year
for year in simulation_years:
    # Extract values for this year from the pivot table
    rainfed_values = []
    irrigated_values = []
    
    # For each metric, extract the value from the pivot table
    for metric in water_balance_metrics:
        if metric in pivot_df.index and ('Rainfed' in pivot_df.columns.levels[1]) and ('Irrigated' in pivot_df.columns.levels[1]):
            # Extract value for rainfed
            rainfed_value = pivot_df.loc[metric, ('Rainfed', year)] if (metric, 'Rainfed', year) in pivot_df else 0
            rainfed_values.append(rainfed_value)
            
            # Extract value for irrigated
            irrigated_value = pivot_df.loc[metric, ('Irrigated', year)] if (metric, 'Irrigated', year) in pivot_df else 0
            irrigated_values.append(irrigated_value)
        else:
            # If data not available in pivot_df, use the multi_season_df directly
            rainfed_row = multi_season_df[(multi_season_df['Year'] == year) & (multi_season_df['Treatment'] == 'Rainfed')]
            irrigated_row = multi_season_df[(multi_season_df['Year'] == year) & (multi_season_df['Treatment'] == 'Irrigated')]
            
            if not rainfed_row.empty and metric in rainfed_row.columns:
                rainfed_values.append(rainfed_row[metric].values[0])
            else:
                rainfed_values.append(0)
                
            if not irrigated_row.empty and metric in irrigated_row.columns:
                irrigated_values.append(irrigated_row[metric].values[0])
            else:
                irrigated_values.append(0)
    
    rainfed_yearly_values.append(rainfed_values)
    irrigated_yearly_values.append(irrigated_values)

# Create the figure with subplots for each year
fig, axes = plt.subplots(1, len(simulation_years), figsize=(18, 6), sharey=True)

# Set width of bars
barWidth = 0.35
r1 = np.arange(len(water_balance_metrics))
r2 = [x + barWidth for x in r1]

# If there's only one year, axes won't be an array
if len(simulation_years) == 1:
    axes = [axes]

# Loop through years and create subplots
for i, year in enumerate(simulation_years):
    ax = axes[i]
    
    # Create bars for each year
    ax.bar(r1, rainfed_yearly_values[i], width=barWidth, label='Rainfed', color='blue', alpha=0.7)
    ax.bar(r2, irrigated_yearly_values[i], width=barWidth, label='Irrigated', color='green', alpha=0.7)
    
    # Add labels and title for each subplot
    ax.set_xlabel('Water Balance Components', fontsize=12)
    ax.set_title(f'{year}', fontsize=14)
    ax.set_xticks([r + barWidth/2 for r in range(len(water_balance_metrics))])
    ax.set_xticklabels([m.replace('Total ', '') for m in water_balance_metrics], rotation=45, ha='right')
    
    # Add value labels on top of bars
    for j, v in enumerate(rainfed_yearly_values[i]):
        ax.text(r1[j], v + 5, f"{v:.1f}", ha='center', fontsize=8)
    
    for j, v in enumerate(irrigated_yearly_values[i]):
        ax.text(r2[j], v + 5, f"{v:.1f}", ha='center', fontsize=8)
    
    # Add grid for better readability
    ax.grid(True, linestyle='--', alpha=0.3, axis='y')
    
    # Only add y-label to the first subplot
    if i == 0:
        ax.set_ylabel('Water (mm)', fontsize=12)

# Add a common legend at the bottom
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=2, fontsize=12)

# Adjust layout to make room for the legend
plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

# Add overall title
fig.suptitle('Multi-Year Seasonal Water Balance: Rainfed vs Irrigated', fontsize=16, y=1.05)

# Save figure
fig_path = os.path.join(charts_dir, 'multi_year_water_balance_comparison.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"Saved multi-year water balance comparison chart to {fig_path}")

plt.show()

## 15. Visualize Cumulative Water Balance Across Years

In [ ]:
# Create a stacked bar chart showing cumulative values over years
fig, ax = plt.subplots(figsize=(12, 8))

# Set up bars positioning
bar_width = 0.35
x = np.arange(len(water_balance_metrics))

# Initialize bottom positions for stacking
rainfed_bottom = np.zeros(len(water_balance_metrics))
irrigated_bottom = np.zeros(len(water_balance_metrics))

# Define colors for different years
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# Plot each year's contribution as a stacked segment
for i, year in enumerate(simulation_years):
    # Get year-specific data
    rainfed_data = rainfed_yearly_values[i]
    irrigated_data = irrigated_yearly_values[i]
    
    # Plot rainfed data
    ax.bar(x, rainfed_data, bar_width, bottom=rainfed_bottom, 
           label=f'Rainfed {year}', color=colors[i], alpha=0.7)
    rainfed_bottom += np.array(rainfed_data)
    
    # Plot irrigated data
    ax.bar(x + bar_width, irrigated_data, bar_width, bottom=irrigated_bottom,
           label=f'Irrigated {year}', color=colors[i], alpha=0.4, hatch='//')
    irrigated_bottom += np.array(irrigated_data)

# Add labels, title and legend
ax.set_xlabel('Water Balance Components', fontsize=14)
ax.set_ylabel('Cumulative Water (mm)', fontsize=14)
ax.set_title(f'Cumulative Water Balance Components ({simulation_years[0]}-{simulation_years[-1]})', fontsize=16)
ax.set_xticks(x + bar_width / 2)
ax.set_xticklabels([m.replace('Total ', '') for m in water_balance_metrics])
ax.legend(ncol=2)

# Add value labels at the top of the stacked bars
for i, v in enumerate(rainfed_bottom):
    ax.text(x[i], v + 10, f"{v:.1f}", ha='center', fontsize=9)
    
for i, v in enumerate(irrigated_bottom):
    ax.text(x[i] + bar_width, v + 10, f"{v:.1f}", ha='center', fontsize=9)

# Add grid for better readability
ax.grid(True, linestyle='--', alpha=0.3, axis='y')

plt.tight_layout()
fig_path = os.path.join(charts_dir, 'cumulative_water_balance.png')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"Saved cumulative water balance chart to {fig_path}")

plt.show()

## 16. Visualize Water Stress (if available)

In [ ]:
# Check for water stress column (handle different possible names)
water_stress_columns = ['Ksw,exp', 'Ksw', 'Ks', 'StExp']
rainfed_stress_col = None
irrigated_stress_col = None

for col in water_stress_columns:
    if col in daily_rainfed.columns and col in daily_irrigated.columns:
        rainfed_stress_col = col
        irrigated_stress_col = col
        break

if rainfed_stress_col and irrigated_stress_col:
    # Extract water stress values for valid dates
    rainfed_stress = [daily_rainfed[rainfed_stress_col].iloc[i] for i in rainfed_indices]
    irrigated_stress = [daily_irrigated[irrigated_stress_col].iloc[i] for i in irrigated_indices]
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot water stress for both simulations
    ax.plot(rainfed_dates, rainfed_stress, 'b-', linewidth=2, label='Rainfed')
    ax.plot(irrigated_dates, irrigated_stress, 'g-', linewidth=2, label='Irrigated')
    
    # Add grid and labels
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Water Stress Coefficient (%)', fontsize=12)
    ax.set_title('Water Stress Effects: Rainfed vs Irrigated', fontsize=14)
    
    # Set y-axis limits
    ax.set_ylim(0, 105)
    
    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    fig.autofmt_xdate()  # Rotate date labels for better readability
    
    # Add horizontal line at 100% (no stress)
    ax.axhline(y=100, color='k', linestyle='--', alpha=0.5)
    ax.text(rainfed_dates[10], 102, 'No Stress Level', fontsize=10)
    
    # Add irrigation events as vertical lines for first year only
    for event in irrigation_events:
        event_date = simulation_periods[0]['planting_date'] + pd.Timedelta(days=event['day'])
        event_datetime = datetime(event_date.year, event_date.month, event_date.day)
        ax.axvline(x=event_datetime, color='c', linestyle='--', alpha=0.5)
    
    # Add legend and annotation
    ax.legend(loc='lower left')
    plt.figtext(0.15, 0.02, "Cyan vertical lines indicate irrigation events (30mm each) in first year", 
               ha="left", fontsize=10, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    
    # Save figure
    fig_path = os.path.join(charts_dir, 'water_stress_comparison_multi_year.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"Saved water stress comparison chart to {fig_path}")
    
    plt.show()
else:
    print("Water stress columns not found in datasets")
    print("Available columns that might indicate stress:")
    stress_candidates = [col for col in daily_rainfed.columns if 'St' in col or 'Ks' in col]
    print(stress_candidates)

## 17. Visualize Soil Water Content (if available)

In [ ]:
# Check for root zone water content column
depletion_columns = ['Dr', 'Wr']
depletion_col = None

for col in depletion_columns:
    if col in daily_rainfed.columns and col in daily_irrigated.columns:
        depletion_col = col
        break

if depletion_col:
    # Extract depletion values for valid dates
    rainfed_dr = [daily_rainfed[depletion_col].iloc[i] for i in rainfed_indices]
    irrigated_dr = [daily_irrigated[depletion_col].iloc[i] for i in irrigated_indices]
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot depletion for both simulations
    ax.plot(rainfed_dates, rainfed_dr, 'b-', linewidth=2, label='Rainfed')
    ax.plot(irrigated_dates, irrigated_dr, 'g-', linewidth=2, label='Irrigated')
    
    # Add grid and labels
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel(f'Root Zone Water Content ({depletion_col})', fontsize=12)
    ax.set_title('Soil Water Content in Root Zone: Rainfed vs Irrigated', fontsize=14)
    
    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    fig.autofmt_xdate()  # Rotate date labels for better readability
    
    # Add irrigation events as vertical lines for first year only
    for event in irrigation_events:
        event_date = simulation_periods[0]['planting_date'] + pd.Timedelta(days=event['day'])
        event_datetime = datetime(event_date.year, event_date.month, event_date.day)
        ax.axvline(x=event_datetime, color='c', linestyle='--', alpha=0.5)
    
    # Add legend and annotation
    ax.legend(loc='upper left')
    plt.figtext(0.15, 0.02, "Cyan vertical lines indicate irrigation events (30mm each) in first year", 
               ha="left", fontsize=10, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    
    # Save figure
    fig_path = os.path.join(charts_dir, 'soil_water_content_comparison_multi_year.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"Saved soil water content comparison chart to {fig_path}")
    
    plt.show()
else:
    print("Root zone water content/depletion column not found in datasets")

## 18. Save Results to CSV Files

In [ ]:
# Create CSV directory
csv_dir = os.path.join(charts_dir, "csv_results")
os.makedirs(csv_dir, exist_ok=True)

# Save daily results
daily_rainfed.to_csv(os.path.join(csv_dir, "daily_rainfed_multi_year.csv"), index=False)
daily_irrigated.to_csv(os.path.join(csv_dir, "daily_irrigated_multi_year.csv"), index=False)

# Save seasonal results
results_rainfed['season'].to_csv(os.path.join(csv_dir, "seasonal_rainfed_multi_year.csv"), index=False)
results_irrigated['season'].to_csv(os.path.join(csv_dir, "seasonal_irrigated_multi_year.csv"), index=False)

print(f"Results saved to {csv_dir}")